In [9]:
from lusidtools.jupyter_tools import toggle_code

"""Inline Valuation Reconciliation Demo

This notebook shows you how to complete an inline reconciliation of two of valuations weighted instruments using different pricing sources.

Attributes
----------
instruments
quotes
recipes
inline reconciliation
holdings
weighted instruments
"""

toggle_code("Hide docstring")

# Inline Valuation Reconciliation

This notebook shows you how to complete an inline reconciliation of two inline valuations of weighted instruments using different pricing sources.

- 2 sets of prices linked to the same 5 client internals are loaded into the quotes store under 2 unique scopes.
- 5 weighted instruments are defined with the same client internals and a weighting (units) set to 1.
- 2 recipes are created, one for each of the pricing scopes.
- An inline reconciliation is completed with 2 inline valuation requests made for the weighted instruments using the two recipes.

**What does inline mean?**

Inline means not existing outside the scope of this notebook. The portfolios, instruments and holdings used in the valuation and reconciliation requests are not saved to LUSID but exist "inline" in this notebook only.

**Why inline?**

This would be useful for instance in a situation where you wanted to quickly compare the valuations produced from different pricing sources for instruments such as emerging market government bonds without loading any data other than quotes.

## 1. Setup LUSID

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import other packages
import os
import pandas as pd
import pytz
from datetime import datetime

# Format dataframes
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define the apis
quotes_api = api_factory.build(lusid.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
reconciliations_api = api_factory.build(lusid.api.ReconciliationsApi)

In [4]:
# Define variables
valuation_date = datetime(year=2022, month=3, day=8, tzinfo=pytz.UTC)
scope = "Ibor"
portfolio_code = "Inline_Recon_NB"

## 2. Load Quotes

### 2.1 Instrument Value Quotes

Load 10 prices into the quotes store with associated instrument 'ClientInternal' identifiers. These will be linked to the weighted instruments created below in Section 3.

In [5]:
# Define instrument prices
source_a_prices = [20, 50, 100, 75, 60]
source_b_prices = [20, 45, 100, 85, 60]

# Define pricing scopes
pricing_scope_a = "pricing_scope_a"
pricing_scope_b = "pricing_scope_b"

# Define empty weighted instrument list
weighted_instruments = []

# Upsert quotes
for i in range(0,5):
    
    # Source A quotes
    quotes_api.upsert_quotes(
        scope = pricing_scope_a,
        request_body = {f"quote_{i+1}": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=f"client_internal_{i+1}",
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid"
                ),
                effective_at = valuation_date
            ),
            metric_value=models.MetricValue(
                value=source_a_prices[i],
                unit="GBP"
            )
        )}
    ) 
    
    # Source B quotes
    quotes_api.upsert_quotes(
        scope = pricing_scope_b,
        request_body = {f"quote_{i+1}": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=f"client_internal_{i+1}",
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid"
                ),
                effective_at = valuation_date
            ),
            metric_value=models.MetricValue(
                value=source_b_prices[i],
                unit="GBP"
            )
        )}
    )  
    
    # Define 5 weighted instruments with the same holding identifiers as the instrument identifiers of the quotes
    if i in range(0, 5):
        weighted_instrument = lusid.WeightedInstrument(
            quantity=1,
            holding_identifier=f"client_internal_{i+1}",
            instrument=models.Equity(
                identifiers= lusid.EquityAllOfIdentifiers(
                    client_internal=f"client_internal_{i+1}",
                ),
                dom_ccy="GBP",
                instrument_type="Equity",
            ), 
        )
        # Append weighted instrument to list
        weighted_instruments.append(weighted_instrument)

## 3. Inline Valuation

### 3.1 Create Valuation Recipes

Define and upsert the two seperate valuation recipes to be used in the inline valuation requests.

These recipes determine that equity quotes (mid) loaded into the quotes store in scopes "pricing_scope_a" or "pricing_scope_b" (depending on which is used) are to be used for valuations.

More details on recipes can be found here: https://support.lusid.com/knowledgebase/article/KA-01895/en-us

In [6]:
# Create recipes
recipe_scope="Inline_Recon_NB"
recipe_a_code="Inline_a_Recon_NB"
recipe_b_code="Inline_b_Recon_NB"


# Define pricing recipe A

configuration_recipe_a = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_a_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=pricing_scope_a,
                quote_type="Price",
                field="mid",
                quote_interval="1D.0D"
            ),
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope='Lusid',
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

# Upsert recipe to LUSID
upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe_a
    )
)


# Define pricing recipe B

configuration_recipe_b = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_b_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=pricing_scope_b,
                quote_type="Price",
                field="mid",
                quote_interval="1D.0D"
            ),
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope='Lusid',
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

# Upsert recipe to LUSID
upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe_b
    )
)

### 3.2 Create Valuation Requests 

Create inline valuation requests for the weighted instruments for the valuation date using the 2 recipes defined above and requesting the following 5 metrics:

1. PV
2. PV Currency
3. Holding Units
4. InstrumentId
5. Quote value

More details on valuations can be found here: https://support.lusid.com/knowledgebase/article/KA-01729/en-us

In [7]:
# Generate valuation request
def generate_valuation_request(recipe_scope, recipe_code, valuation_effectiveAt, instruments):

    # Create the valuation request
    valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=recipe_scope, code=recipe_code
        ),
        metrics=[
            models.AggregateSpec("Valuation/PV", "Value"),
            models.AggregateSpec("Valuation/PV/Ccy", "Value"),
            models.AggregateSpec("Holding/Units", "Value"),
            models.AggregateSpec("Analytic/default/InstrumentTag", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
        ],
        group_by=["Analytic/default/InstrumentTag"],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
        instruments=instruments
    )

    return valuation_request

### 3.3 Reconcile Inline Valuation Responses

- Complete an inline reconciliation using 2 inline valuation requests created with the weighted instrument list and the 2 valuation recipes defined above.

- Reformat the inline reconciliation response to combine left side, right side and difference data into a single reconciliation table.

The output shows a difference in PV for the 'client_internal_2' and 'client_internal_4' weighted instruments which matches up with the price difference between the 2 sources referenced in the recipes.

These 5 instruments are valued using 2 different pricing sources and reconciled against each other without the need to create instruments, portfolios or holdings in LUSID. The only data written to LUSID are the prices written to the quotes store and so inline valuations and reconciliations are really useful for completing What-If analysis where you don't want to flood your LUSID environment with excess data.

In [8]:
# Reconcile weighted instruments
reconciliation_respose = reconciliations_api.reconcile_inline(
    inline_valuations_reconciliation_request=models.InlineValuationsReconciliationRequest(
        left=generate_valuation_request(recipe_scope, recipe_a_code, valuation_date, weighted_instruments),
        right=generate_valuation_request(recipe_scope, recipe_b_code, valuation_date, weighted_instruments)))

# Format response into dataframe including left side, right side and difference data
def format_inline_recon_response(reconciliation_respose):
    recs_diff_df = pd.DataFrame(reconciliation_respose.diff)
    recs_diff_df = recs_diff_df.rename(columns={"Valuation/PV" : "PV Diff", "Analytic/default/InstrumentTag" : "ClientInternal"})
    recs_diff_df = recs_diff_df.drop(columns=['Valuation/PV/Ccy','Holding/Units', 'Quotes/Price'])
    recs_left_df = pd.DataFrame(reconciliation_respose.left.data)
    recs_left_df = recs_left_df.rename(columns={"Valuation/PV/Ccy" : "Ccy", "Valuation/PV" : "Left PV", "Holding/Units" : "Units", "Analytic/default/InstrumentTag" : "ClientInternal", "Quotes/Price" : "Left Price"})
    recs_right_df = pd.DataFrame(reconciliation_respose.right.data)
    recs_right_df = recs_right_df.rename(columns={"Valuation/PV/Ccy" : "Ccy", "Valuation/PV" : "Right PV", "Holding/Units" : "Units", "Analytic/default/InstrumentTag" : "ClientInternal", "Quotes/Price" : "Right Price"})
    recs_df = pd.merge(recs_left_df, recs_right_df, on='ClientInternal', how='left')
    recs_df = pd.merge(recs_df, recs_diff_df, on='ClientInternal', how='left')
    recs_df = recs_df.drop(columns=['Units_y','Ccy_y'])
    recs_df = recs_df.rename(columns={"Ccy_x" : "Ccy", "Units_x" : "Units"})
    recs_df = recs_df[['ClientInternal', 'Ccy', 'Units', 'Left Price', 'Left PV', 'Right Price', 'Right PV', 'PV Diff']]
    return recs_df

recon_output = format_inline_recon_response(reconciliation_respose)

# Convert df to csv
recon_output.to_csv('inline_rec.csv')

# Output results dataframe
recon_output

,ClientInternal,Ccy,Units,Left Price,Left PV,Right Price,Right PV,PV Diff
0,client_internal_1,GBP,1.0,20.0,20.0,20.0,20.0,0.0
1,client_internal_2,GBP,1.0,50.0,50.0,45.0,45.0,-5.0
2,client_internal_3,GBP,1.0,100.0,100.0,100.0,100.0,0.0
3,client_internal_4,GBP,1.0,75.0,75.0,85.0,85.0,10.0
4,client_internal_5,GBP,1.0,60.0,60.0,60.0,60.0,0.0
